# Form the NB Regression Random Intercept Model: Simulated set

Using Block update of beta and MM-update. We will use the file "fit_old.jl"

Without having to compute the gradient and hessian of variance components vector, we finish in 67 iterations and 9 seconds.

Next we may try joint estimation using Newton's + IPOPT. 

In [1]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using SpecialFunctions

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.9
variance_component_2 = 0.1

r = 10
p = 0.7
μ = r * (1-p) * inv(p)

#var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

┌ Info: Precompiling GLMCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc]
└ @ Base loading.jl:1317


10000-element Vector{Vector{Float64}}:
 [2.0, 1.0, 2.0, 3.0, 1.0]
 [8.0, 5.0, 3.0, 5.0, 7.0]
 [7.0, 4.0, 4.0, 11.0, 7.0]
 [4.0, 3.0, 7.0, 4.0, 9.0]
 [10.0, 11.0, 10.0, 1.0, 8.0]
 [4.0, 4.0, 3.0, 4.0, 3.0]
 [9.0, 6.0, 5.0, 4.0, 4.0]
 [6.0, 1.0, 3.0, 6.0, 0.0]
 [4.0, 7.0, 3.0, 3.0, 6.0]
 [5.0, 5.0, 3.0, 6.0, 2.0]
 [7.0, 2.0, 3.0, 2.0, 3.0]
 [3.0, 2.0, 2.0, 1.0, 4.0]
 [7.0, 6.0, 2.0, 7.0, 6.0]
 ⋮
 [2.0, 3.0, 1.0, 4.0, 2.0]
 [8.0, 3.0, 6.0, 2.0, 7.0]
 [2.0, 1.0, 3.0, 5.0, 7.0]
 [2.0, 3.0, 1.0, 1.0, 5.0]
 [7.0, 3.0, 4.0, 4.0, 11.0]
 [2.0, 3.0, 6.0, 2.0, 6.0]
 [2.0, 3.0, 1.0, 1.0, 2.0]
 [8.0, 9.0, 6.0, 3.0, 5.0]
 [5.0, 6.0, 2.0, 3.0, 2.0]
 [5.0, 6.0, 6.0, 9.0, 5.0]
 [0.0, 2.0, 1.0, 4.0, 3.0]
 [6.0, 6.0, 8.0, 3.0, 6.0]

In [2]:
d = NegativeBinomial(10) # set r here
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
@show gcm.β
@show gcm.Σ
@show gcm.data[1].d.r

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -121027.88891374407 39999
2 -121027.88891374407 -121027.88891374407 39999
initializing variance components using MM-Algorithm
gcm.Σ = [0.6979507824600143, 0.08742459243806242]
gcm.β = [1.5151931648552381]
gcm.Σ = [0.6979507824600143, 0.08742459243806242]
(gcm.data[1]).d.r = 10.000197918145874


10.000197918145874

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [3]:
GLMCopula.loglikelihood!(gcm, true, true)

-118480.31878364169

## Fit_old (not updating r) works

If true $r$ is not 1, setting the correct $r$ in initialize function will make it work too.

In [12]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 500, mehrotra_algorithm="yes", warm_start_init_point="yes", hessian_approximation = "exact"))

gcm.β = [1.5151931648552381]
gcm.Σ = [0.6979257382428836, 0.08740945654307972]
gcm.β = [1.5151931648552381]
gcm.Σ = [0.6979244525871305, 0.0874084929729934]
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:    

In [13]:
@show β_true
@show gcm.β
@show gcm.Σ
@show variance_component_1, variance_component_2
@show gcm.∇β
@show gcm.data[1].d
@show GLMCopula.loglikelihood!(gcm, true, true);

β_true = 1.4552872326068422
gcm.β = [1.45381327887095]
gcm.Σ = [0.8537921429784839, 0.12204795378602154]
(variance_component_1, variance_component_2) = (0.9, 0.1)
gcm.∇β = [-2.7448491024095745e-5]
(gcm.data[1]).d = NegativeBinomial{Float64}(r=10.0, p=0.5)
GLMCopula.loglikelihood!(gcm, true, true) = -118247.93818277148


## fit_old (updating r)

In [21]:
d = NegativeBinomial() # r is 1
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)] # V[1] isn't positive definite!!!!
#     A = randn(n, n)
#     V = [A'*A, Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
@show gcm.β
@show gcm.Σ
@show gcm.data[1].d.r


initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -120610.51602032165 39999
2 -120610.51602032165 -120610.51602032165 39999
initializing variance components using MM-Algorithm
gcm.Σ = [0.5136083074694615, 4.486990696535601e-5]
gcm.β = [1.5151931648552381]
gcm.Σ = [0.5136083074694615, 4.486990696535601e-5]
(gcm.data[1]).d.r = 8.00000194402719


8.00000194402719

In [9]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 50, 
    mehrotra_algorithm="yes", warm_start_init_point="yes", hessian_approximation = "exact"))


gcm.β = [1.5151931648552381]
new_d = NegativeBinomial{Float64}(r=5.194588346551785, p=0.5)
gcm.Σ = [0.3227068245489785, 4.0492171291795376e-7]
gcm.β = [1.5151931648552381]
new_d = NegativeBinomial{Float64}(r=5.194589750941404, p=0.5)
gcm.Σ = [0.3227057747820965, 3.441164992974993e-7]
new_d = NegativeBinomial{Float64}(r=5.194591403162907, p=0.5)
gcm.Σ = [0.3227048868480921, 2.924421032472634e-7]
new_d = NegativeBinomial{Float64}(r=5.194593385827047, p=0.5)

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number o

LoadError: InterruptException:

In [64]:
@show β_true
@show gcm.β
@show gcm.Σ
@show variance_component_1, variance_component_2
@show gcm.∇β
@show gcm.∇Σ
@show gcm.data[1].d
@show GLMCopula.loglikelihood!(gcm, true, true);

β_true = 1.4552872326068422
gcm.β = [1.4515827907066625]
gcm.Σ = [0.7227698167857666, 0.04825752326740178]
(variance_component_1, variance_component_2) = (0.9, 0.1)
gcm.∇β = [-0.0001557668805842738]
gcm.∇Σ = [0.001905348967284226, 0.020403430535679945]
(gcm.data[1]).d = NegativeBinomial{Float64}(r=9.000049930238733, p=0.5)
GLMCopula.loglikelihood!(gcm, true, true) = -118252.41344944645
